<a href="https://colab.research.google.com/github/danielsaggau/IR_LDC/blob/main/hyperparameter_tuning_classification_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebooks provides some hyperparameter tuning

# Set up Dataset

In [7]:
#load packages
!pip install transformers


In [ ]:
#load data 
!pip install datasets
from datasets import load_dataset
dataset=load_dataset("lex_glue","scotus")
train_dataset=dataset['train']
train_dataset = train_dataset.shard(index=1, num_shards=10)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
padding="max_length"

tokenizer = AutoTokenizer.from_pretrained('danielsaggau/bregman_1.5', use_fast=True)

def preprocess_function(examples):
      return tokenizer(examples["text"], truncation=True, padding=padding)

tokenized_data = train_dataset.map(
      preprocess_function,
      batched=True,
      desc="tokenizing the entire dataset")

# Set up Trainer 

load model


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('danielsaggau/bregman_1.5',num_labels=14)

Compute Metric Function


In [15]:
def compute_metrics(eval_pred):
  metric1 = load_metric("f1")
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  micro1 = metric1.compute(predictions=predictions, references=labels, average="micro")["f1"]
  macro1 = metric1.compute(predictions=predictions, references=labels, average="macro")["f1"]
  return { "f1-micro": micro1, "f1-macro": macro1} 

Training Arguments

In [29]:
training_args = TrainingArguments(
    output_dir='scotus_max_linear',
    learning_rate=3e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy="epoch",
    evaluation_strategy="epoch",
 #   push_to_hub=True,
    metric_for_best_model="f1-micro",
   # fp16=True,
#    report_to="wandb",
    greater_is_better=True,
    lr_scheduler_type='linear',
 #   run_name="max",
    load_best_model_at_end = True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [24]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8) # fp16

In [ ]:
from transformers import Trainer, EarlyStoppingCallback
trainer = Trainer(
    compute_metrics=compute_metrics,
    args=training_args,
    train_dataset=tokenized_data,
    tokenizer=tokenizer,
    data_collator=data_collator,    
    model_init=model_init,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
      )

# Define hyperparameter space 

In [47]:
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0.01,0.05),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [2,3,4,6,8]),
        "num_train_epochs": trial.suggest_categorical("num_train_epochs", [4,5,6,7, 10,15]),
        "lr_scheduler_type": trial.suggest_categorical("lr_scheduler_type", ['linear', 'cosine', 'polynomial','constant', 'constant_with_warmup'])
    }

In [42]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained('danielsaggau/bregman_1.5', num_labels=14)


In [ ]:
#hide_output
#!pip install optuna
best_run = trainer.hyperparameter_search(
    n_trials=20, direction="maximize", hp_space=hp_space)

[I 2022-12-11 22:10:54,162] A new study created in memory with name: no-name-b9936bc6-457a-4fa4-ae98-d4e0d230b5e7
Trial: {'learning_rate': 4.3617949466081894e-05, 'weight_decay': 0.04718274235109328, 'per_device_train_batch_size': 4, 'num_train_epochs': 6, 'lr_scheduler_type': 'cosine'}
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--danielsaggau--bregman_1.5/snapshots/363ae19253237bb845fc9861c93ac6033414e92d/config.json
Model config LongformerConfig {
  "_name_or_path": "danielsaggau/bregman_1.5",
  "architectures": [
    "LongformerModel"
  ],
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    128,
    128,
    128,
    128,
    128,
    128
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "cls_token_id": 101,
  "eos_token_id": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "id2label": {
    "0": "LABEL_0",
    "1

Epoch,Training Loss,Validation Loss


Initializing global attention on CLS token...
Initializing global attention on CLS token...
